In [1]:
import numpy as np
import pandas as pd
from datetime import date
import time
import scipy
import scipy.signal
import os, os.path
import matplotlib
import matplotlib.pyplot as plt

import sys
sys.path.append('/Users/hn/Documents/00_GitHub/Ag/NASA/Python_codes/')
import NASA_core as nc
# import NASA_plot_core.py as rcp

# Set up Directories

In [2]:
data_dir = "/Users/hn/Documents/01_research_data/NASA/VI_TS/05_SG_TS/"
ML_data_dir = "/Users/hn/Documents/01_research_data/NASA/ML_data/"

# Set other parameters

In [3]:
idx="EVI"

# Read Train Labels and IDs

In [4]:
limitCrops_nonExpert_votes = pd.read_csv(ML_data_dir + "limitCrops_nonExpert_votes.csv")
print (limitCrops_nonExpert_votes.shape)
limitCrops_nonExpert_votes.head(2)

(1315, 16)


,Form,Question,ID,Hossein,Supriya,Kirti,Mike,Min,Vote,NDVI_TS_Name,CropTyp,Irrigtn,DataSrc,Acres,LstSrvD,county
0,31,3,53189_WSDA_SF_2016,Single Crop,Single Crop,Single Crop,none,none,single,AdamBenton2016_53189_WSDA_SF_2016.png,barley,center pivot,wsda,132,2016/09/20 00:00:00,Adams
1,31,4,101534_WSDA_SF_2018,Single Crop,Double Crop,Single Crop,none,none,single,FranklinYakima2018_101534_WSDA_SF_2018.png,barley,center pivot,wsda,3,2018/08/14 00:00:00,Franklin


# Read TS files

In [5]:
file_names = ["SG_Walla2015_" + idx + "_JFD.csv", "SG_AdamBenton2016_" + idx + "_JFD.csv", 
              "SG_Grant2017_" + idx + "_JFD.csv", "SG_FranklinYakima2018_" + idx + "_JFD.csv"]

data=pd.DataFrame()

for file in file_names:
    curr_file=pd.read_csv(data_dir + file)
    curr_file['human_system_start_time'] = pd.to_datetime(curr_file['human_system_start_time'])
    
    # These data are for 3 years. The middle one is the correct one
    all_years = sorted(curr_file.human_system_start_time.dt.year.unique())
    if len(all_years)==3 or len(all_years)==2:
        proper_year = all_years[1]
    elif len(all_years)==1:
        proper_year = all_years[0]

    curr_file = curr_file[curr_file.human_system_start_time.dt.year==proper_year]
    data=pd.concat([data, curr_file])

data.reset_index(drop=True, inplace=True)
data.loc[data[idx]<0, idx]=0
data.head(2)

,ID,human_system_start_time,EVI
0,135073_WSDA_SF_2015,2015-01-10,0.054429
1,135073_WSDA_SF_2015,2015-01-20,0.051731


# Filter the train fields TS

In [6]:
limitCrops_nonExpert_votes_IDs = list(limitCrops_nonExpert_votes.ID.unique())
data = data[data.ID.isin(limitCrops_nonExpert_votes_IDs)]
data.reset_index(drop=True, inplace=True)

In [7]:
len(data.ID.unique())

1315

In [8]:
for curr_ID in data.ID.unique():
    crr_fld=data[data.ID==curr_ID].copy()
    crr_fld.reset_index(drop=True, inplace=True)
    # crr_fld['human_system_start_time'] = pd.to_datetime(crr_fld['human_system_start_time'])
    SFYr = crr_fld.human_system_start_time.dt.year.unique()[0]
    fig, ax = plt.subplots();
    fig.set_size_inches(10, 2.5)
    ax.grid(False);
    ax.plot(crr_fld['human_system_start_time'], crr_fld[idx], 
            c ='dodgerblue', linewidth=5)

    ax.axis("off")
    # ax.set_xlabel('time'); # , labelpad = 15
    # ax.set_ylabel(idx, fontsize=12); # , labelpad = 15
    # ax.tick_params(axis = 'y', which = 'major');
    # ax = plt.gca()
    # ax.axes.xaxis.set_visible(False)
    # ax.axes.yaxis.set_visible(False)

    left = crr_fld['human_system_start_time'][0]
    right = crr_fld['human_system_start_time'].values[-1]
    ax.set_xlim([left, right]); # the following line alsow orks
    ax.set_ylim([-0.005, 1]); # the following line alsow orks


    crop_count_letter = limitCrops_nonExpert_votes[limitCrops_nonExpert_votes.ID==curr_ID]["Vote"].values[0]
    plot_path = "/Users/hn/Documents/01_research_data/NASA/ML_data/limitCrops_nonExpert_images_" + idx + "/"
    os.makedirs(plot_path, exist_ok=True)
    fig_name = plot_path + crop_count_letter + "_" + curr_ID +'.jpg'
    plt.savefig(fname = fig_name, dpi=100, bbox_inches='tight', facecolor="w")
    plt.close('all')
    # ax.legend(loc = "upper left");

# NDVI

In [9]:
idx="NDVI"

In [10]:
file_names = ["SG_Walla2015_" + idx + "_JFD.csv", "SG_AdamBenton2016_" + idx + "_JFD.csv", 
              "SG_Grant2017_" + idx + "_JFD.csv", "SG_FranklinYakima2018_" + idx + "_JFD.csv"]

data=pd.DataFrame()

for file in file_names:
    curr_file=pd.read_csv(data_dir + file)
    curr_file['human_system_start_time'] = pd.to_datetime(curr_file['human_system_start_time'])
    
    # These data are for 3 years. The middle one is the correct one
    all_years = sorted(curr_file.human_system_start_time.dt.year.unique())
    if len(all_years)==3 or len(all_years)==2:
        proper_year = all_years[1]
    elif len(all_years)==1:
        proper_year = all_years[0]

    curr_file = curr_file[curr_file.human_system_start_time.dt.year==proper_year]
    data=pd.concat([data, curr_file])

data.reset_index(drop=True, inplace=True)
data.loc[data[idx]<0, idx]=0
data.head(2)

,ID,human_system_start_time,NDVI
0,135073_WSDA_SF_2015,2015-01-10,0.115126
1,135073_WSDA_SF_2015,2015-01-20,0.111097


In [11]:
limitCrops_nonExpert_votes_IDs = list(limitCrops_nonExpert_votes.ID.unique())
data = data[data.ID.isin(limitCrops_nonExpert_votes_IDs)]
data.reset_index(drop=True, inplace=True)
len(data.ID.unique())

1315

In [12]:
for curr_ID in data.ID.unique():
    crr_fld=data[data.ID==curr_ID].copy()
    crr_fld.reset_index(drop=True, inplace=True)
    # crr_fld['human_system_start_time'] = pd.to_datetime(crr_fld['human_system_start_time'])
    SFYr = crr_fld.human_system_start_time.dt.year.unique()[0]
    fig, ax = plt.subplots();
    fig.set_size_inches(10, 2.5)
    ax.grid(False);
    ax.plot(crr_fld['human_system_start_time'], crr_fld[idx], 
            c ='dodgerblue', linewidth=5)

    ax.axis("off")
    # ax.set_xlabel('time'); # , labelpad = 15
    # ax.set_ylabel(idx, fontsize=12); # , labelpad = 15
    # ax.tick_params(axis = 'y', which = 'major');
    # ax = plt.gca()
    # ax.axes.xaxis.set_visible(False)
    # ax.axes.yaxis.set_visible(False)

    left = crr_fld['human_system_start_time'][0]
    right = crr_fld['human_system_start_time'].values[-1]
    ax.set_xlim([left, right]); # the following line alsow orks
    ax.set_ylim([-0.005, 1]); # the following line alsow orks


    crop_count_letter = limitCrops_nonExpert_votes[limitCrops_nonExpert_votes.ID==curr_ID]["Vote"].values[0]
    plot_path = "/Users/hn/Documents/01_research_data/NASA/ML_data/limitCrops_nonExpert_images_" + idx + "/"
    os.makedirs(plot_path, exist_ok=True)
    fig_name = plot_path + crop_count_letter + "_" + curr_ID +'.jpg'
    plt.savefig(fname = fig_name, dpi=100, bbox_inches='tight', facecolor="w")
    plt.close('all')
    # ax.legend(loc = "upper left");
    
print (plot_path)

/Users/hn/Documents/01_research_data/NASA/ML_data/limitCrops_nonExpert_images_NDVI/


In [13]:
crr_fld.head(2)

,ID,human_system_start_time,NDVI
0,99791_WSDA_SF_2018,2018-01-10,0.135647
1,99791_WSDA_SF_2018,2018-01-20,0.165818
